# Machine Learning Basics - Regression

This notebook provides a practical and theoretical introduction to **Regression analysis**.

**Objectives:**
- Understand the theory behind regression
- Explore different regression techniques
- Apply regression to real data
- Build interactive visualizations and models


## What is Regression?

Regression is a fundamental statistical and machine learning technique used to model and analyze the relationship between a dependent variable (_target_) and one or more independent variables (_features_). It helps us understand how the typical value of the dependent variable changes when any one of the independent variables is varied.

**Why is regression important?**
- Predict future outcomes (e.g., sales, revenue)
- Identify key drivers of business metrics
- Support data-driven decision making
- Detect trends and patterns

**Possible Usecases**
Regression can be used for example for:
- Forecasting
- Resource planning
- Performance analysis
- Cost prediction
- etc.

## Types of Regression

There are several types of regression techniques, each suited for different scenarios:

- **Linear Regression:** Models the relationship between two variables by fitting a straight line. Useful for simple, linear relationships.
- **Multiple Linear Regression:** Extends linear regression to multiple features.
- **Polynomial Regression:** Models non-linear relationships by fitting a polynomial curve.
- **Regularized Regression (Ridge, Lasso):** Adds penalties to prevent overfitting and improve generalization.
- **Logistic Regression:** Used for predicting categories (e.g., success or failure), not continuous values.

In this notebook, we will focus on linear and multiple regression, and demonstrate how to apply them to real data.

# Use Case
As a supermarket manager, you want to understand what are the main drivers for your sales. You have noticed that sales have been going well, but in order to increase your branch's revenues even more, you want to target your offers towards the products that drive sales. How can you do that? 


## The Dataset: Supermarket Sales

We will use the [Supermarket Sales dataset](https://www.kaggle.com/datasets/faresashraf1001/supermarket-sales) from Kaggle. This dataset contains sales records for a supermarket, including product, branch, customer, and sales details.

In [ ]:
import pandas as pd

dataset_path = '../../data/SuperMarketAnalysis.csv'

# Load the dataset
df = pd.read_csv(dataset_path)
df.head()

## Data Dictionary

Let's review the main columns in the dataset:

- **Invoice ID:** Unique identifier for each transaction
- **Branch:** Branch of the supermarket (A, B, or C)
- **City:** City where the branch is located
- **Customer type:** Member or Normal
- **Gender:** Customer gender
- **Product line:** Product category
- **Unit price:** Price per product
- **Quantity:** Number of products purchased
- **Tax 5%:** Tax amount
- **Total:** Total price including tax
- **Date/Time:** Date and time of purchase
- **Payment:** Payment method
- **COGS:** Cost of goods sold
- **Gross margin percentage:** Gross margin
- **Gross income:** Gross income from the sale
- **Rating:** Customer rating

We will use these features to explore regression techniques.

## Frame the Supermarket Sales Problem as a Regression task
From the above data dictionary, our **dependent variable** (target) will be `Sales`, as we want to increase revenues. While **independent variables** are inputs that help explain or predict the target. These can be:
- Product-related, `Product line`, `Quantity`, `Unit price`
- Customer-related, `Gender`, `Customer type`, `City` etc.
- Transaction-related, `Date` ,`Time` etc.

How are these independent variables useful?
<details>
        <summary>Answer: Click to show</summary>
    Each independent variable helps explain variation of sales in different ways. For example, `Product line` captures different sales patterns across categories, `City` accounts for regional differences in customer demand, store size, or local economy. `Date` and `Time` capture temporal effects, like sales trends over days or peak shopping hours. Together, these features give the regression model multiple perspectives on _why_ a given transaction's total might be higher or lower.
</details>


The **expected output** would be:
1) The prediction of the supermarket sales based on the influence of the independent variable
2) An equation that models the relationship between these variables

Why would we want an equation?
<details>
        <summary>Answer: Click to show</summary>
    The equation will show the mathematical relationship of the independent variables with respect to the dependent variables. The coefficient will also be useful to determine how much each input influences our target `Sales`.
</details>



## Exploratory Data Analysis (EDA)
Before diving into modeling, it is important to have first a clear understanding of our dataset (data formats, distributions, missing values, etc.). Let's first check its schema and what are the datatypes that it includes.

In [ ]:
# Display basic information about the dataset
df.info()

In [ ]:
# Check for distribution of numerical features
df.describe()

From a high level perspective, we notice that we have in total 17 columns and 1000 data points. The Y variable (what we have to predict) is the total Sales (`Sales`), i.e. based on the features captured, how much is the total sales going to be like?

Before that though, let's check the distribution of `Total Sales`:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize distribution of Total sales
plt.figure(figsize=(8,4))
sns.histplot(df['Sales'], bins=30, kde=True)
plt.title('Distribution of Total Sales')
plt.xlabel('Total sales')
plt.ylabel('Frequency')
plt.show()

From this chart we notice the Total distribution of sales. In the x-axis we have the number of total sales, while in the y-axis we have how often these sales have happened. It tells us:
- how sales values are spread out (e.g. do most transactions fall in the low-medium range, or are there many high-value purchases?)
- presence of skewness or outliers (we notice that the distribution is skewed to the right, meaning that majority of the sales were small purchase and very few large)
- customer puchasing behaviour (the shape can help reveal whether spending habits vary widely).

Understanding the distribution of the dependent variable is key in the exploratory phase as it helps:
- decide if transformations are needed for modeling (e.g. as our sales distribution is skewed to the right, we might consider to take the `log(Sales)` so that large transactions would not dominate the fit)
- guide business insights, for example if we know that people make mostly small purchases, then it would also affect how inventory strategy gets optimized 

And now let's do the same by product line:

In [ ]:
# Visualize sales by product line
plt.figure(figsize=(10,4))
sns.boxplot(x='Product line', y='Sales', data=df)
plt.title('Sales by Product Line')
plt.xticks(rotation=45)
plt.show()

We notice that overall they are quite similar with only a couple of outliers in `Sports and travel`, `Foods and beverages`, and `Fashion accessories`. Not only this, but `Fashion accessories` seems to have the least sales in general.

# Linear Regressions

The equation looks like this:

$$
\text{y} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_p x_p + \varepsilon\
$$

where:
- 𝑦: Dependent variable (the target we want to predict, e.g., sales).  
- β0: Intercept (baseline value of 𝑦 when all variables are 0).  
- 𝛽1,𝛽2,…,𝛽𝑝: Coefficients (show how much 𝑦 changes when each x increases by 1, holding others constant).  
- x1​,x2​,…,xp: Independent variables (features/variables).  
- ε: Error term (captures noise and variation not explained by the model).  


## Simple Linear Regression

Simple linear regression models the relationship between two variables by fitting a straight line. In business, it can be used to predict sales based on a **single feature**, such as unit price or quantity.

**Example use cases:**
- Predicting sales from advertising spend
- Estimating revenue from number of customers

Let's build a simple linear regression model to predict total `Sales` based on `Unit price`.



In [ ]:
# Simple Linear Regression: Predict Total from Unit price
from sklearn.linear_model import LinearRegression
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets

# Prepare data
X = df[['Unit price']].values
y = df['Sales'].values

model = LinearRegression()
model.fit(X, y)

# Predict
y_pred = model.predict(X)

# Interactive plot
@widgets.interact(unit_price=widgets.FloatSlider(min=float(df['Unit price'].min()), max=float(df['Unit price'].max()), step=0.5, value=float(df['Unit price'].mean()), description='Unit price:'))
def plot_regression(unit_price):
    pred = model.predict(np.array([[unit_price]]))[0]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Unit price'], y=df['Sales'], mode='markers', name='Data'))
    fig.add_trace(go.Scatter(x=df['Unit price'], y=y_pred, mode='lines', name='Regression Line'))
    fig.add_trace(go.Scatter(x=[unit_price], y=[pred], mode='markers', marker=dict(size=12, color='red'), name='Prediction'))
    fig.update_layout(title='Simple Linear Regression: Total vs Unit price', xaxis_title='Unit price', yaxis_title='Total')
    fig.show()
    print(f'Predicted Total for Unit price {unit_price:.2f}: {pred:.2f}')

## Multiple Linear Regression

Multiple linear regression models the relationship between a target variable and **two or more** features. This allows us to capture more complex relationships and improve prediction accuracy.

**Business relevance:**
- Predicting sales using multiple factors (e.g., unit price, quantity, branch, product line)
- Understanding the impact of different variables on business outcomes

Let's build a multiple regression model to predict total `Sales` using several features.

In [ ]:
# Multiple Linear Regression: Predict Total from multiple features
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Select features (excluding target and identifiers)
features = ['Unit price', 'Quantity', 'Branch', 'Product line', 'Gender', 'Customer type']

Before moving on to the modeling phase, a necessary step is needed: Feature Encoding

In [ ]:
# Encode categorical variables
df_encoded = pd.get_dummies(df[features], drop_first=True)


Encoding categorical variables is necessary because regression models require **numerical input**. However, real-world data often include categorical variables (e.g., `Product line`). These must be converted into numerical format because:
- Models cannot understand text or categories natively
- Numeric reoresebtatuibs allow models to learn relationship between features and their target (`Sales`)

There are several types of encoding methods depending on the type and number of categories, some are: 

1) **One hot encoding**
- Creates a new binary column for each category 
- Best for nomincal (unordered) categories 


```
Color: ["red", "green", "blue"]
→ One-hot encoded:
Red | Green | Blue
 1  |   0   |  0
 0  |   1   |  0
 0  |   0   |  1
```

2) **Label encoding**
- Assigns a unique integer to each category.
- Can mislead models into thinking there's an ordinal relationship.
- Often used for tree-based models, or truly ordinal variables.

```
Color: ["red", "green", "blue"]
→ Label encoded: [2, 1, 0]
```


3) **Ordinal encoding**
/ Smilar to label encoding but used when categories have a natural oder (e.g. small < medium < large)

```
Size: ["small", "medium", "large"]
→ Ordinal encoded: [0, 1, 2]
```

In our use case we used `pd.get_dummies()` which is a form of one-hot encoding. It in fact converted our categorical variables into binary indicator variables - one column for each category, with `1` indicating presence and `0` absence.

Before:

In [ ]:
df['Branch'].head()

After one-hot encoding:

In [ ]:
df_encoded[['Branch_Cairo', 'Branch_Giza']].head()

We use one-hot encoding in linear regression when categorical variables are present, because the model can only handle numeric inputs. 

Instead of assigning arbitrary numbers (which would wrongly imply an order), one-hot encoding creates binary indicator variables that let the model learn a separate coefficient for each category without introducing false ordinal relationships.

Let's go back to our modeling now!

In [ ]:
X = df_encoded
y = df['Sales']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_multi = LinearRegression()
model_multi.fit(X_train, y_train)

y_pred = model_multi.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.2f}')
print(f'R^2 Score: {r2:.2f}')

- Mean Squared Error (MSE) measures the average of the square of the errors, i.e. how far your model's predictions are from the actual values (the lower the better). In our case, on average our predictions are off by about square root of MSE units (80 units).
- R squared tells you the proportion of variance in the target, i.e. the % of the bariance in the target that is explained by the model (the closer to 1 the better). In our case, our model can explain the data quite well.

In [ ]:
# Let's compare performances on training and test sets
y_train_pred = model_multi.predict(X_train)
y_test_pred = model_multi.predict(X_test)

# Mean Squared Error
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print("Train MSE:", mse_train)
print("Test MSE:", mse_test)

As you can see, our MSE can be improved. Both MSE are high, meaning that our model is most likely _underfitting_ and we should consider more flexible models, such as polynomial features, non-linear models, or improving our features with feature engineering.


What is underfitting?
<details>
        <summary>Answer: Click to show</summary>
    Underfitting happens when a model is too simple to capture the underlying patterns in the data. It fails to learn enough from the training set, leading to high error in both training and test data
</details>


## Summary and Next Steps

In this notebook, you have learned:
- The theory and business relevance of regression
- How to apply simple and multiple linear regression
- How to use regularization to improve model performance
- How to interactively explore regression models using real data

**Next steps:**
- Try different features and targets. Feel free to edit the current notebook with additional features and test out how adding more features influences the model prediction!
- Explore non-linear regression and tree-based models (more info in [./05_advanced_regression.ipynb](./05_advanced_regression.ipynb))

For more advanced topics, consider learning about:
- Model evaluation and selection
- Cross-validation
- Feature engineering
- Machine learning pipelines

Thank you for following this regression tutorial!